In [1]:
from typing import Dict, Tuple, Union, List, Any
from fuzzingbook.Fuzzer import RandomFuzzer

In [2]:
random_fuzzer = RandomFuzzer()
random_fuzzer.fuzz()

'+(09:> !%-=8\':>+2<,)", !:'

In [3]:
print(RandomFuzzer.__init__.__doc__)

Produce strings of `min_length` to `max_length` characters
           in the range [`char_start`, `char_start` + `char_range`)


In [4]:
random_fuzzer = RandomFuzzer(min_length=10, max_length=20, char_start=97, char_range=26)
random_fuzzer.fuzz()

'ansifpbtiz'

In [5]:
from fuzzingbook.Fuzzer import PrintRunner

print_runner = PrintRunner()
random_fuzzer.run(print_runner)

mbzfvhvfqxeliffactpz


('mbzfvhvfqxeliffactpz', 'UNRESOLVED')

In [6]:
from fuzzingbook.Fuzzer import ProgramRunner

cat = ProgramRunner("cat")
random_fuzzer.run(cat)

(CompletedProcess(args='cat', returncode=0, stdout='sjjmqlnqdjouqtzfk', stderr=''),
 'PASS')

In [7]:
import random


def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for _ in range(string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [8]:
fuzzer()

'=/=316$;"(418.1$1)\'* 7/<$>90"?"5".*9(&\'$>\'4%8\';**:>78;:"-45 +4).5-1?3<?."7!=?+<10.1((,3'

In [9]:
fuzzer(1000, ord("a"), 26)

'awfxvgwanhtqraqyemiurkkxofuhddwmhqyxjxbxjzqlbntljajkqiwxjatsamukkkhwbbgnwfmuxsvzsglkupavyypygcbffebrlgpzinlgdhoksmruavgldmvopzmannplxznckjxdbnwgfdkpljhcabjeefecdrgyroywohcyriicthksyiuklmrxbeiubftottksttmvfholxjecjuwhxzconwbpbslnejjyljbixfqinqadsmzztqigehkypybrcedbxtlphyzzcdhyflprbbspfbschqbivkjelbbqsjbmdzvuztcwzaqsuttiutumwykgkxqmmihhjdpcjafnzaxurbzbimhblgrtbbdhyymxopntdsvlqyooswypkgljumgppazzxmqyskwxraxnkottqmewwyjxjwaqfhgbzppumnkahfivepapvjsgmygkvojgdqaqldqmtufssaofkiolocrdxwtlzzbzawjacbwamzfgrkqtncwohxvwyldydvrgxoykdffalqighvshqyyausbwngrsecbmxpzgyjmxnhxnwwwzuoqujqfwmaqkrtjcunmvqljlehelcxdpbsxcybdhlpdkzwjfsidolwavrrbkligshiijimnhdjkyatckpytunkkaqvuublyvzrqqhfwysfqldzyuifjnoffqgxzbagzsqundehtnyireqkxzdesrwndoofmdosthxniascthgdbarszurdrskfycvxbbayjtkheuztdxvmvooqcnituhsrxwefenlfxfjwzezoykdtyxomjyinjhgyyphefivrxbjudzhbczxzlwzyaoqsubqynnipqdozckgijclerxqsjuddtsektwrjbrckswidjxyufysmhfnuzltvjecvlxdfpxordxyoedyyytzwolwqyptlnipgjkkqvxnrnhmedhvvzqrxxppwvsdgrhahq'

In [10]:
import os
import tempfile

basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

/tmp/tmpfdant_21/input.txt


In [11]:
data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)

contents = open(FILE).read()
print(contents)
assert contents == data

"'>+;%9,<7/60=/-?25.$=#>:-3 '=/9?+:(#.)&?< ,%7-4*?)> 7;0<!-7.&*) .09/.>*'.


In [12]:
import subprocess

program = "bc"
with open(FILE, "w") as f:
    f.write("2+2\n")
result = subprocess.run(
    [program, FILE],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    universal_newlines=True,
)

In [13]:
result.stdout

'4\n'

In [14]:
result.returncode

0

In [15]:
result.stderr

In [16]:
trials = 100
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        result = subprocess.run(
            [program, FILE],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            universal_newlines=True,
        )

        runs.append((data, result))

In [17]:
sum(1 for (data, result) in runs if result.stderr == "")

0

In [18]:
errors = [(data, result) for (data, result) in runs if result.stderr != ""]
(first_data, first_result) = errors[0]
print(repr(first_data))
print(first_result.stderr)

'9<; $**2:3$=,9$?:68)67>*&4*+)\'=/89=.05(  8;#"9/*3:$42=%4$)>-7.$,,?+4#$09*#9=3+,,<+\'+*3.=8=1+?4'
None


In [19]:
[
    result.stderr
    for (data, result) in runs
    if result.stderr
    and "illegal character" not in result.stderr
    and "parse error" not in result.stderr
    and "syntax error" not in result.stderr
]

[]

In [20]:
def crash_if_too_long(s):
    buffer = "Thursday"
    if len(s) > len(buffer):
        raise ValueError

In [21]:
from fuzzingbook.Fuzzer import ExpectError

trials = 100
with ExpectError():
    for i in range(trials):
        s = fuzzer()
        crash_if_too_long(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/2508798955.py", line 7, in <module>
    crash_if_too_long(s)
  File "/tmp/ipykernel_29385/2784561514.py", line 4, in crash_if_too_long
    raise ValueError
ValueError (expected)


In [22]:
def hang_if_n_space(s):
    i = 0
    while True:
        if i < len(s):
            if s[i] == " ":
                break
        i += 1

In [23]:
from fuzzingbook.Fuzzer import ExpectTimeout

trials = 100
with ExpectTimeout(2):
    for _ in range(trials):
        s = fuzzer()
        hang_if_n_space(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/1808899220.py", line 7, in <module>
    hang_if_n_space(s)
  File "/tmp/ipykernel_29385/2837961466.py", line 3, in hang_if_n_space
    while True:
  File "/home/harshil/.pyenvs/fuzzing/lib/python3.11/site-packages/fuzzingbook/Timeout.py", line 147, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)


In [24]:
def collapse_if_too_large(s):
    if int(s) > 1000:
        raise ValueError

In [25]:
long_number = fuzzer(100, ord("0"), 10)
print(long_number)

43397786215634951133116660514401868402669133606071663051307034509478020


In [26]:
with open("program.c", "w") as f:
    f.write(
        """
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """
    )

In [27]:
from fuzzingbook.bookutils import print_file

print_file("program.c")

#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    

In [28]:
!clang -fsanitize=address -g -o program program.c

In [29]:
!./program 99; echo $?

42


In [30]:
!./program 110

==29523==ERROR: AddressSanitizer: heap-buffer-overflow on address 0x60b00000015e at pc 0x558351af7f4c bp 0x7ffefae06dd0 sp 0x7ffefae06dc8
READ of size 1 at 0x60b00000015e thread T0
    #0 0x558351af7f4b in main /home/harshil/concurrency-testing-fuzzing/program.c:12:16
    #1 0x7fec93e6b249 in __libc_start_call_main csu/../sysdeps/nptl/libc_start_call_main.h:58:16
    #2 0x7fec93e6b304 in __libc_start_main csu/../csu/libc-start.c:360:3
    #3 0x558351a3a300 in _start (/home/harshil/concurrency-testing-fuzzing/program+0x20300) (BuildId: a7defe265a60f982a1ddea1c01cc23792b1596ff)

0x60b00000015e is located 10 bytes to the right of 100-byte region [0x60b0000000f0,0x60b000000154)
allocated by thread T0 here:
    #0 0x558351abd14e in __interceptor_malloc (/home/harshil/concurrency-testing-fuzzing/program+0xa314e) (BuildId: a7defe265a60f982a1ddea1c01cc23792b1596ff)
    #1 0x558351af7ebf in main /home/harshil/concurrency-testing-fuzzing/program.c:7:17
    #2 0x7fec93e6b249 in __libc_start_call_

In [31]:
!rm -fr program program.*

In [32]:
secrets = (
    "<space for reply>"
    + fuzzer(100)
    + "<secret-certificate>"
    + fuzzer(100)
    + "<secret-key>"
    + fuzzer(100)
    + "<other-secrets>"
)

In [33]:
uninitialized_memory_marker = "deadbeef"
while len(secrets) < 2048:
    secrets += uninitialized_memory_marker

In [34]:
def heartbeat(reply: str, length: int, memory: str) -> str:
    # Store reply in memory
    memory = reply + memory[len(reply) :]

    # Send back heartbeat
    s = ""
    for i in range(length):
        s += memory[i]
    return s

In [35]:
heartbeat("potato", 6, memory=secrets)

'potato'

In [36]:
heartbeat("bird", 4, memory=secrets)

'bird'

In [37]:
heartbeat("hat", 500, memory=secrets)

'hatace for reply>)+-34-?%2+*!<.">>\',=.-*#\'58.+,&\'<!:)3;,:5:\'*\'(6=#/-(1:810>!?>92!41&>1$294<secret-certificate>&":/52)\'+ #(024"4$131$)+-5>)74%>!%25?)*8:"=<secret-key>\'=/;/970/&!8-"81!=$%.2%,1#;**\'\'!3\',%:<other-secrets>deadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdea'

In [38]:
with ExpectError():
    for _ in range(10):
        s = heartbeat(fuzzer(), random.randint(1, 500), memory=secrets)
        assert not s.find(uninitialized_memory_marker)
        assert not s.find("secret")

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/4023609742.py", line 4, in <module>
    assert not s.find(uninitialized_memory_marker)
AssertionError (expected)


In [39]:
airport_codes: Dict[str, str] = {
    "YVR": "Vancouver",
    "JFK": "New York-JFK",
    "CDG": "Paris-Charles de Gaulle",
    "CAI": "Cairo",
    "LED": "St. Petersburg",
    "PEK": "Beijing",
    "HND": "Tokyo-Haneda",
    "AKL": "Auckland",
}

In [40]:
airport_codes["YVR"]

'Vancouver'

In [41]:
"AKL" in airport_codes

True

In [42]:
def code_repOK(code: str) -> bool:
    assert len(code) == 3, "Airport code must have three characters" + repr(code)
    for c in code:
        assert c.isalpha(), "Non-Letter in airport code: " + repr(code)
        assert c.isupper(), "Lowercase letter in airport code: " + repr(code)
    return True

In [43]:
assert code_repOK("SEA")

In [44]:
def airport_codes_repOK():
    for code in airport_codes:
        assert code_repOK(code)
    return True

In [45]:
with ExpectError():
    assert airport_codes_repOK()

In [46]:
airport_codes["YMML"] = "Melbourne"
with ExpectError():
    assert airport_codes_repOK()

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/3670898385.py", line 3, in <module>
    assert airport_codes_repOK()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_29385/480627665.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_29385/2970345604.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters" + repr(code)
           ^^^^^^^^^^^^^^
AssertionError: Airport code must have three characters'YMML' (expected)


In [47]:
def add_new_airport(code: str, city: str) -> None:
    assert code_repOK(code)
    airport_codes[code] = city

In [48]:
with ExpectError():  # For BER, ExpectTimeout would be more appropriate
    add_new_airport("BER", "Berlin")

In [49]:
with ExpectError():
    add_new_airport("London-Heathrow", "LHR")

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/1427835309.py", line 2, in <module>
    add_new_airport("London-Heathrow", "LHR")
  File "/tmp/ipykernel_29385/2655039924.py", line 2, in add_new_airport
    assert code_repOK(code)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_29385/2970345604.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters" + repr(code)
           ^^^^^^^^^^^^^^
AssertionError: Airport code must have three characters'London-Heathrow' (expected)


In [50]:
def add_new_airport_2(code: str, city: str) -> None:
    assert code_repOK(code)
    assert airport_codes_repOK()
    airport_codes[code] = city
    assert airport_codes_repOK()

In [51]:
with ExpectError():
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")

Traceback (most recent call last):
  File "/tmp/ipykernel_29385/1096579022.py", line 2, in <module>
    add_new_airport_2("IST", "Istanbul Yeni Havalimanı")
  File "/tmp/ipykernel_29385/2099116665.py", line 3, in add_new_airport_2
    assert airport_codes_repOK()
           ^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_29385/480627665.py", line 3, in airport_codes_repOK
    assert code_repOK(code)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_29385/2970345604.py", line 2, in code_repOK
    assert len(code) == 3, "Airport code must have three characters" + repr(code)
           ^^^^^^^^^^^^^^
AssertionError: Airport code must have three characters'YMML' (expected)


In [ ]:
class RedBlackTree:
    def repOK(self):
        assert self.rootHasNoParent()
        assert self.rootIsBlack()
        assert self.rootNodesHaveOnlyBlackChildren()
        assert self.treeIsAcyclic
        assert self.parentsAreConsistent()
        return True

    def rootIsBlack(self):
        if self.parent is None:
            assert self.color == BLACK
        return True

    def add_element(self, elem):
        assert self.repOK()
        ##Insertion algo

    def delete_element(self, elem):
        assert self.repOK()
        assert self.repOK()

In [53]:
typed_airport_codes: Dict[str, str] = {
    "YVR": "Vancouver",  # etc
}

In [54]:
typed_airport_codes[1] = "First"